# Laboratorio 4 Malware families
## Security Data Science
### Universidad del valle de Guatemala

Abner Ivan Garcia - 21285

Oscar Esteba Donis - 21610

# Parte 1
## Creación del dataset

In [2]:
import os
import pefile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score

In [9]:
dataset = [] # Lista que contendrá las instancias de nuestro dataset

for filename in os.listdir("MALWR"): # Iteramos sobre los archivos en la carpeta MALWR
    filepath = os.path.join("MALWR", filename) # Obtenemos la ruta completa del archivo

    try:
        pe = pefile.PE(filepath) # Cargamos el archivo PE en memoria
        file_size = os.path.getsize(filepath) # Obtenemos el tamaño del archivo en bytes
        creation_time = os.path.getctime(filepath) # Obtenemos la fecha de creación del archivo
        modification_time = os.path.getmtime(filepath) # Obtenemos la fecha de modificación del archivo

        sections = [section.Name.decode().strip('\x00') for section in pe.sections] # Obtenemos las secciones del archivo
        num_sections = len(sections) # Obtenemos el número de secciones del archivo

        # Llamadas a funciones importadas
        imported_functions = [] # Lista que contendrá las funciones importadas
        for entry in pe.DIRECTORY_ENTRY_IMPORT: # Iteramos sobre las entradas de la tabla de importación
            for function in entry.imports: # Iteramos sobre las funciones importadas
                imported_functions.append(function.name.decode()) # Agregamos la función a la lista

        embedded_strings = [] # Lista que contendrá las cadenas de texto incrustadas
        try:
            for s in pe.strings: # Iteramos sobre las cadenas de texto incrustadas
                if isinstance(s, bytes): # Si la cadena es de tipo bytes la decodificamos
                    embedded_strings.append(s.decode(errors='ignore')) # Agregamos la cadena a la lista
                else:
                    embedded_strings.append(s) # Agregamos la cadena a la lista
        except AttributeError:
            pass

        instance = [file_size, creation_time, modification_time, num_sections, sections, imported_functions, embedded_strings] # Creamos una instancia con los atributos del archivo
        dataset.append(instance) # Agregamos la instancia a nuestro dataset
    except Exception as e:
        print(f"{str(e)}")

columns = ["Tamaño del archivo", "Tiempo de creación", "Tiempo de modificación", "Número de secciones", "Secciones del archivo", "Llamadas a funciones importadas", "Cadenas de texto incrustadas"]
df = pd.DataFrame(dataset, columns=columns)
print("Encabezados de nuestro Dataset creado:")
print(df.head())

'DOS Header magic not found.'
Encabezados de nuestro Dataset creado:
   Tamaño del archivo  Tiempo de creación  Tiempo de modificación  \
0              285184        1.741100e+09            1.535140e+09   
1               15360        1.741100e+09            1.535140e+09   
2              295424        1.741100e+09            1.535140e+09   
3               15360        1.741100e+09            1.535140e+09   
4               16896        1.741100e+09            1.535140e+09   

   Número de secciones          Secciones del archivo  \
0                    3             [UPX0, UPX1, UPX2]   
1                    3         [.text, .rdata, .data]   
2                    3            [UPX0, UPX1, .rsrc]   
3                    3         [.text, .rdata, .data]   
4                    4  [.text, .rdata, .data, .rsrc]   

                     Llamadas a funciones importadas  \
0  [RegSaveKeyA, BitBlt, LoadLibraryA, ExitProces...   
1  [CreateFileA, LocalAlloc, Sleep, CreateThread,...   
2  [R

In [5]:
df.head()

,Tamaño del archivo,Tiempo de creación,Tiempo de modificación,Número de secciones,Secciones del archivo,Llamadas a funciones importadas,Cadenas de texto incrustadas
0,285184,1.741100e+09,1.535140e+09,3,"[UPX0, UPX1, UPX2]","[RegSaveKeyA, BitBlt, LoadLibraryA, ExitProces...",[]
1,15360,1.741100e+09,1.535140e+09,3,"[.text, .rdata, .data]","[CreateFileA, LocalAlloc, Sleep, CreateThread,...",[]
2,295424,1.741100e+09,1.535140e+09,3,"[UPX0, UPX1, .rsrc]","[RegSaveKeyA, BitBlt, LoadLibraryA, GetProcAdd...",[]
3,15360,1.741100e+09,1.535140e+09,3,"[.text, .rdata, .data]","[CreateFileA, LocalAlloc, Sleep, CreateThread,...",[]
4,16896,1.741100e+09,1.535140e+09,4,"[.text, .rdata, .data, .rsrc]","[GetModuleHandleA, GetWindowsDirectoryA, GetMo...",[]


In [11]:
df.describe()

,Tamaño del archivo,Tiempo de creación,Tiempo de modificación,Número de secciones
count,13.000000,1.300000e+01,1.300000e+01,13.000000
mean,104487.384615,1.741100e+09,1.535140e+09,3.076923
std,139879.808920,3.591415e-02,3.094660e+00,0.277350
min,14336.000000,1.741100e+09,1.535140e+09,3.000000
25%,15360.000000,1.741100e+09,1.535140e+09,3.000000
50%,15360.000000,1.741100e+09,1.535140e+09,3.000000
75%,285184.000000,1.741100e+09,1.535140e+09,3.000000
max,344576.000000,1.741100e+09,1.535140e+09,4.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Tamaño del archivo               13 non-null     int64  
 1   Tiempo de creación               13 non-null     float64
 2   Tiempo de modificación           13 non-null     float64
 3   Número de secciones              13 non-null     int64  
 4   Secciones del archivo            13 non-null     object 
 5   Llamadas a funciones importadas  13 non-null     object 
 6   Cadenas de texto incrustadas     13 non-null     object 
dtypes: float64(2), int64(2), object(3)
memory usage: 856.0+ bytes


## Exploración y pre-procesamiento de datos

In [13]:
df['Cantidad de cadenas incrustadas'] = df['Cadenas de texto incrustadas'].apply(len)
print(df['Cantidad de cadenas incrustadas'].value_counts())

Cantidad de cadenas incrustadas
0    13
Name: count, dtype: int64


In [14]:
df.drop('Cadenas de texto incrustadas', axis=1, inplace=True)
df.drop('Cantidad de cadenas incrustadas', axis=1, inplace=True)

In [15]:
df.rename(columns={
    'Tamaño del archivo': 'file_size',
    'Tiempo de creación': 'creation_time',
    'Tiempo de modificación': 'modification_time',
    'Número de secciones': 'num_sections',
    'Secciones del archivo': 'file_sections',
    'Llamadas a funciones importadas': 'imported_function_calls',
    'Cadenas de texto incrustadas': 'embedded_strings',
    'Cantidad de cadenas incrustadas': 'num_embedded_strings'
}, inplace=True)
print("Encabezado del Dataset con Nombres de Columnas Actualizados:")
df.head()

Encabezado del Dataset con Nombres de Columnas Actualizados:


,file_size,creation_time,modification_time,num_sections,file_sections,imported_function_calls
0,285184,1.741100e+09,1.535140e+09,3,"[UPX0, UPX1, UPX2]","[RegSaveKeyA, BitBlt, LoadLibraryA, ExitProces..."
1,15360,1.741100e+09,1.535140e+09,3,"[.text, .rdata, .data]","[CreateFileA, LocalAlloc, Sleep, CreateThread,..."
2,295424,1.741100e+09,1.535140e+09,3,"[UPX0, UPX1, .rsrc]","[RegSaveKeyA, BitBlt, LoadLibraryA, GetProcAdd..."
3,15360,1.741100e+09,1.535140e+09,3,"[.text, .rdata, .data]","[CreateFileA, LocalAlloc, Sleep, CreateThread,..."
4,16896,1.741100e+09,1.535140e+09,4,"[.text, .rdata, .data, .rsrc]","[GetModuleHandleA, GetWindowsDirectoryA, GetMo..."


In [17]:
df_processed = df.copy()

# Tokenizar las columnas de listas
mlb_sections = MultiLabelBinarizer()
mlb_imported_functions = MultiLabelBinarizer()

# Tokenización de las secciones del archivo
sections_encoded = pd.DataFrame(mlb_sections.fit_transform(df_processed['file_sections']), columns=mlb_sections.classes_, index=df_processed.index)

# Tokenización de las llamadas a funciones importadas
imported_functions_encoded = pd.DataFrame(mlb_imported_functions.fit_transform(df_processed['imported_function_calls']), columns=mlb_imported_functions.classes_, index=df_processed.index)

df_processed = pd.concat([df_processed, sections_encoded, imported_functions_encoded], axis=1)
df_processed.drop(['file_sections', 'imported_function_calls'], axis=1, inplace=True)
scaler = MinMaxScaler()
df_processed[['file_size', 'creation_time', 'modification_time']] = scaler.fit_transform(df_processed[['file_size', 'creation_time', 'modification_time']])
print("Encabezado del Dataset ya Procesado:")
df_processed.head()

Encabezado del Dataset ya Procesado:


,file_size,creation_time,modification_time,num_sections,.data,.rdata,.rsrc,.text,UPX0,UPX1,...,strcpy,strlen,strstr,time,wcscat,wcscmp,wcscpy,wcslen,wcsrchr,wcstombs
0,0.820155,0.000000,0.625,3,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0.003101,0.080887,0.000,3,1,1,0,1,0,0,...,1,1,1,1,1,1,1,1,1,1
2,0.851163,0.164867,0.625,3,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0.003101,0.255180,0.000,3,1,1,0,1,0,0,...,1,1,1,1,1,1,1,1,1,1
4,0.007752,0.329884,0.000,4,1,1,1,1,0,0,...,1,1,1,1,0,0,0,0,0,0


# Parte 2 
## Implementación de algoritmos
### Kmeans